In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


### 1) Importing libraries and dataset

In [ ]:
# path = '/content/drive/MyDrive/nlp/Aggression_Imbalance_DS.csv'

In [1]:
!nvidia-smi

Sat Jul  9 17:00:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets
!pip install ray[tune]

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

In [6]:
# either put path(if defined) in train value here

aggressions = load_dataset('csv', data_files={'train': 'Aggression_Imbalance_DS.csv'})

aggressions.set_format(type='pandas') # Converting from HF's dataset type to pandas dataframe
df = aggressions['train'][:]
df.head()

Using custom data configuration default-adf6a5ae51016630


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-adf6a5ae51016630/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,Sentence,Category
0,what a wicket !!!!!!!!! woohooooo #indvsuae #w...,NAG
1,And because of u people there is a boundary cr...,OAG
2,Arandhuti Roy is a person who do not believe i...,NAG
3,Saurabh Shinde wht does that mean at least he ...,CAG
4,NBCC has a very good order book but will that ...,NAG


In [11]:
df['Category'].value_counts()

NAG    5602
CAG    4240
OAG    2821
Name: Category, dtype: int64

In [7]:
# Label encoding

dt = {'NAG':0, 'CAG':1, 'OAG':2}
df['Category'].unique()
labels = map(lambda x: dt[x], df['Category'])
labels = list(labels)
labels = pd.Series(labels)

df['Label'] = labels
labs = np.array(labels)
df.head(5)

,Sentence,Category,Label
0,what a wicket !!!!!!!!! woohooooo #indvsuae #w...,NAG,0
1,And because of u people there is a boundary cr...,OAG,2
2,Arandhuti Roy is a person who do not believe i...,NAG,0
3,Saurabh Shinde wht does that mean at least he ...,CAG,1
4,NBCC has a very good order book but will that ...,NAG,0


### 2) Tokenizing and making train, test and valid splits

In [8]:
model_ckpt = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [9]:
texts = list(df['Sentence'])
labels = list(df['Label'])

In [10]:
train_size=0.8

# In the first step we will split the data in training and remaining dataset
train_texts, X_rem, train_labels, y_rem = train_test_split(texts, labels, train_size=0.8, random_state=43)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
valid_texts, test_texts, valid_labels, test_labels = train_test_split(X_rem, y_rem, test_size=0.5, random_state=43)

print(str(np.shape(train_texts))), print(str(np.shape(train_labels)))
print(str(np.shape(valid_texts))), print(str(np.shape(valid_labels)))
print(str(np.shape(test_texts))), print(str(np.shape(test_labels)))

(10130,)
(10130,)
(1266,)
(1266,)
(1267,)
(1267,)


(None, None)

In [16]:
# USE this only for hyperparameter tuning
# In this step we will split the train data (15 %) and validation data (50%) only for hyperparameter tuning 


_texts, hyp_texts, _labels, hyp_labels = train_test_split(train_texts, train_labels, test_size=0.15, random_state=43)
print(str(np.shape(hyp_texts))), print(str(np.shape(hyp_labels)))

_texts2, hyp_v_texts, _labels2, hyp_v_labels = train_test_split(valid_texts, valid_labels, test_size=0.5, random_state=43)
print(str(np.shape(hyp_v_texts))), print(str(np.shape(hyp_v_labels)))



(1520,)
(1520,)
(633,)
(633,)


(None, None)

In [17]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True)
hyp_encodings = tokenizer(hyp_texts, truncation=True, padding=True)
hyp_v_encodings = tokenizer(hyp_v_texts, truncation=True, padding=True)

In [18]:
import torch
# tensors
class AggressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [19]:
train_dataset = AggressionDataset(train_encodings, train_labels)
valid_dataset = AggressionDataset(valid_encodings, valid_labels)
hyp_dataset = AggressionDataset(hyp_encodings, hyp_labels)
hyp_v_dataset = AggressionDataset(hyp_v_encodings, hyp_v_labels)

In [ ]:
# Run for out of memory cases

# import gc

# gc.collect()
# torch.cuda.empty_cache()

### 3) Custom Trainer with weighted loss objective

In [20]:
from torch import nn

class CustomTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    device = torch.device('cuda')
    inputs.to(device)
    labels = inputs.get("labels")
    # forward pass
    outputs = model(**inputs)
    logits = outputs.get("logits")
    # compute custom loss (suppose one has 3 labels with different weights)
    loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.21, 0.33, 0.46]).to(device))
    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

### 4) Hyperparameter Tuning (using PBT)

In [21]:
from ray.tune.schedulers import PopulationBasedTraining
import ray
from ray import tune
from ray.tune import CLIReporter

In [22]:
from transformers import AutoModelForSequenceClassification

# num_labels = 3
# device = torch.device('cuda')
# if torch.cuda.is_available() else 'cpu'
# print(device)
# model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

def get_my_model():
  num_labels = 3
  device = torch.device('cuda')
  return (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

In [23]:
from datasets import load_metric

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  metric = load_metric("f1")
  return metric.compute(predictions=preds, references=labels, average="macro")

In [24]:
# See this for more stoppers
# https://docs.ray.io/en/latest/tune/api_docs/stoppers.html

from ray.tune import Stopper

class CustomStopper(tune.Stopper):
        def __init__(self):
            self.should_stop = False

        def __call__(self, trial_id, result):
            max_iter = 5 
            if not self.should_stop and result["training_iteration"] >= max_iter:
                self.should_stop = True

        def stop_all(self):
            return self.should_stop

stopper = CustomStopper()

In [25]:
from transformers import Trainer, TrainingArguments
from ray.tune.examples.pbt_transformers.utils import build_compute_metrics_fn

# task_name = 'rte'

training_args = TrainingArguments(
        output_dir=".",
        learning_rate=1.5e-5,  # config
        do_train=True,
        do_eval=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        num_train_epochs=2,  # config
        max_steps=-1,
        per_device_train_batch_size=16,  # config
        per_device_eval_batch_size=16,  # config
        warmup_steps=0,
        weight_decay=0.1,  # config
        logging_dir="./logs",
        skip_memory_metrics=True,
        report_to="none",
        # metric_for_best_model=metric_name,

    )

trainer = CustomTrainer(model_init=get_my_model, args=training_args,
        compute_metrics = compute_metrics,
        train_dataset = hyp_dataset,
        eval_dataset = hyp_v_dataset,
        tokenizer = tokenizer)

tune_config = {
        "per_device_train_batch_size": tune.choice([16, 32]),
        "per_device_eval_batch_size": 16,
        "num_train_epochs": tune.choice([3, 4, 5, 6]),
        # "max_steps": 1 if smoke_test else -1,  # Used for smoke test.
    }

scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="eval_f1",
        mode="max",
        perturbation_interval=1,
        hyperparam_mutations={
            "weight_decay": tune.uniform(0.0, 0.3),
            "learning_rate": tune.uniform(1e-5, 5e-5),
            "per_device_train_batch_size": [16, 32],
        },
    )

reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_device_train_batch_size": "train_bs/gpu",
            "num_train_epochs": "num_epochs",
        },
        metric_columns=["eval_f1", "eval_loss", "epoch", "training_iteration"],
    )


ress = trainer.hyperparameter_search(
        hp_space=lambda _: tune_config,
        backend="ray",
        n_trials=4,
        resources_per_trial={"cpu": 0, "gpu": 1}, # can be parallelized , https://docs.ray.io/en/latest/tune/tutorials/tune-resources.html
        scheduler=scheduler,
        keep_checkpoints_num=1,
        checkpoint_score_attr="training_iteration",
        stop=stopper,
        progress_reporter=reporter,
        local_dir="~/ray_results/",
        name="tune_transformer_pbt",
        log_to_file=True,
    )

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transfo

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
creating metadata file for /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', '

== Status ==
Current time: 2022-07-09 17:04:30 (running for 00:00:00.21)
Memory usage on this node: 4.4/25.5 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/4 CPUs, 1.0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc            |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+----------------+-----------+-------------+----------------+--------------|
| _objective_32814_00000 | RUNNING  | 172.28.0.2:528 | 0.285214  | 3.92798e-05 |             16 |            6 |
| _objective_32814_00001 | PENDING  |                | 0.0468056 | 1.62398e-05 |             16 |            3 |
| _objective_32814_00002 | PENDING  |                | 0.2

(_objective pid=528) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
(_objective pid=528) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=528) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=528) Some weights of RobertaForSequenceClassification were not initialized from the model checkpoi

...

 33%|███▎      | 95/285 [01:19<02:39,  1.19it/s]
(_objective pid=747) 
  0%|          | 0/40 [00:00<?, ?it/s]
(_objective pid=747) 
  5%|▌         | 2/40 [00:00<00:05,  7.40it/s]
(_objective pid=747) 
  8%|▊         | 3/40 [00:00<00:07,  5.22it/s]
(_objective pid=747) 
 10%|█         | 4/40 [00:00<00:07,  4.51it/s]
(_objective pid=747) 
 12%|█▎        | 5/40 [00:01<00:08,  4.19it/s]
2022-07-09 17:07:57,315	WARNING tune.py:683 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(_objective pid=747) 
 15%|█▌        | 6/40 [00:01<00:08,  4.01it/s]
(_objective pid=747) 
 18%|█▊        | 7/40 [00:01<00:08,  3.89it/s]
(_objective pid=747) 
 20%|██        | 8/40 [00:01<00:08,  3.83it/s]
(_objective pid=747) 
 22%|██▎       | 9/40 [00:02<00:08,  3.78it/s]
(_objective pid=747) 
 25%|██▌       | 10/40 [00:02<00:08,  3.75it/s]
(_objective pid=747) 
 28%|██▊   

== Status ==
Current time: 2022-07-09 17:07:59 (running for 00:03:28.88)
Memory usage on this node: 7.2/25.5 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/4 CPUs, 1.0/1 GPUs, 0.0/15.0 GiB heap, 0.0/7.5 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/tune_transformer_pbt
Number of trials: 3/3 (1 PAUSED, 1 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------+
| Trial name             | status   | loc            |   w_decay |          lr |   train_bs/gpu |   num_epochs |   eval_f1 |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+----------------+-----------+-------------+----------------+--------------+-----------+-------------+---------+----------------------|
| _objective_32814_00001 | RUNNING  | 172.28.0.2:747 | 0.0468056 | 1.62398e-05 |             16 |

(_objective pid=747) 
 32%|███▎      | 13/40 [00:03<00:07,  3.70it/s]
(_objective pid=747) 
 35%|███▌      | 14/40 [00:03<00:07,  3.69it/s]
(_objective pid=747) 
 38%|███▊      | 15/40 [00:03<00:06,  3.68it/s]
(_objective pid=747) 
 40%|████      | 16/40 [00:04<00:06,  3.66it/s]
(_objective pid=747) 
 42%|████▎     | 17/40 [00:04<00:06,  3.66it/s]
(_objective pid=747) 
 45%|████▌     | 18/40 [00:04<00:06,  3.65it/s]
(_objective pid=747) 
 48%|████▊     | 19/40 [00:04<00:05,  3.65it/s]
(_objective pid=747) 
 50%|█████     | 20/40 [00:05<00:05,  3.65it/s]
(_objective pid=747) 
 52%|█████▎    | 21/40 [00:05<00:05,  3.65it/s]
(_objective pid=747) 
 55%|█████▌    | 22/40 [00:05<00:04,  3.65it/s]
(_objective pid=747) 
 57%|█████▊    | 23/40 [00:05<00:04,  3.67it/s]
(_objective pid=747) 
 60%|██████    | 24/40 [00:06<00:04,  3.67it/s]
(_objective pid=747) 
 62%|██████▎   | 25/40 [00:06<00:04,  3.67it/s]
(_objective pid=747) 
 65%|██████▌   | 26/40 [00:06<00:03,  3.68it/s]
(_objective pid=747)

(_objective pid=747) {'eval_loss': 0.8660064339637756, 'eval_f1': 0.5617679680567881, 'eval_runtime': 11.223, 'eval_samples_per_second': 56.402, 'eval_steps_per_second': 3.564, 'epoch': 1.0}


                                                
100%|██████████| 40/40 [00:10<00:00,  4.20it/s]
                                               
2022-07-09 17:08:12,415	ERROR tune.py:743 -- Trials did not complete: [_objective_32814_00000, _objective_32814_00001, _objective_32814_00002]
2022-07-09 17:08:12,417	INFO tune.py:748 -- Total run time: 222.21 seconds (208.88 seconds for the tuning loop).
2022-07-09 17:08:12,420	WARNING tune.py:754 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
